In [30]:
import mxnet.ndarray as nd
import mxnet.autograd as ag
features = 2
examples = 100
X = nd.random_normal(shape=(examples, features))
true_w = [1.4, 0.4]
true_b = 1.2
Y = true_w[0]*X[:, 0]+true_w[1]*X[:, 1]+true_b
Y += 0.1*nd.random_normal(shape=Y.shape)
X, Y

(
 [[ 0.8997614  -0.68561006]
  [-0.438372    0.10900874]
  [-0.24237496  0.0744305 ]
  [-0.878746    0.7767273 ]
  [ 0.3479402  -0.88589776]
  [-1.0920666   0.4524119 ]
  [-1.2167361  -1.0090067 ]
  [ 0.3935671   1.0544698 ]
  [-0.24898548  1.989548  ]
  [ 1.199108    1.6460917 ]
  [ 0.2609435  -0.9187904 ]
  [-0.21924148  1.20272   ]
  [ 0.70329595  0.9105568 ]
  [-0.38999596 -0.05547183]
  [ 0.1282245   2.37668   ]
  [-0.19772758  2.0854106 ]
  [ 0.5125954  -1.5752442 ]
  [ 0.5586102   0.19766884]
  [ 2.1481535  -0.19020747]
  [ 0.27743885 -0.9118177 ]
  [-1.9265597   0.20007446]
  [-0.9868158   0.3224166 ]
  [-0.1010764   0.02885438]
  [-0.05574904 -0.7670688 ]
  [-0.14989796 -1.0163515 ]
  [ 0.37264898  1.4637082 ]
  [ 0.16197115 -0.27533087]
  [-0.6607965  -0.0401484 ]
  [ 1.5821487  -1.3058236 ]
  [-0.8360042  -0.82413524]
  [ 1.0865679  -1.4994327 ]
  [-1.4236078   0.02837564]
  [ 0.8418826  -0.10568757]
  [ 0.04687965  0.37848872]
  [ 1.2343034  -0.9906856 ]
  [ 0.03354292  1.

In [31]:
from mxnet import gluon as gl 
batch_size=10 
dataset=gl.data.ArrayDataset(X,Y)
data_iter=gl.data.DataLoader(dataset,batch_size,shuffle=True)
for X,y in data_iter:
    print(X,y)
    break


[[ 1.2296662   0.24398364]
 [-1.2167361  -1.0090067 ]
 [ 0.3935671   1.0544698 ]
 [-1.0920666   0.4524119 ]
 [-0.6607965  -0.0401484 ]
 [ 1.8646097  -0.818179  ]
 [ 0.68439865 -0.04890326]
 [-0.9868158   0.3224166 ]
 [ 0.37264898  1.4637082 ]
 [-0.33219483  1.2063538 ]]
<NDArray 10x2 @cpu(0)> 
[ 2.9920168  -0.8370806   1.978476   -0.26468825  0.15046698  3.5660322
  2.1197708  -0.09305978  2.2404492   1.1247495 ]
<NDArray 10 @cpu(0)>


In [32]:
from mxnet.gluon import nn
net=nn.Sequential()
net.add(nn.Dense(1))
net.initialize()
from mxnet.gluon import loss as gloss
loss=gloss.L2Loss();
from mxnet import gluon
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':.02 })

In [33]:
from mxnet import autograd
epochs = 12
for e in range(1, epochs+1):
    total_loss = 0
    for X, y in data_iter:
        with autograd.record():
            output = net(X)
            los = loss(output, y)
        los.backward()
        trainer.step(batch_size)
        total_loss += nd.sum(los).asscalar()
    print('epoch: %d ,avg loss: %f' % (e, total_loss/examples))

epoch: 1 ,avg loss: 1.522305
epoch: 2 ,avg loss: 1.033264
epoch: 3 ,avg loss: 0.701900
epoch: 4 ,avg loss: 0.479606
epoch: 5 ,avg loss: 0.329168
epoch: 6 ,avg loss: 0.227403
epoch: 7 ,avg loss: 0.157927
epoch: 8 ,avg loss: 0.110691
epoch: 9 ,avg loss: 0.078360
epoch: 10 ,avg loss: 0.055886
epoch: 11 ,avg loss: 0.040773
epoch: 12 ,avg loss: 0.030088


In [35]:
dense=net[0]
true_w,dense.weight.data(),true_b,dense.bias.data()

([1.4, 0.4], 
 [[1.232498   0.41518816]]
 <NDArray 1x2 @cpu(0)>, 1.2, 
 [1.0903422]
 <NDArray 1 @cpu(0)>)

In [36]:
help(trainer.step)

Help on method step in module mxnet.gluon.trainer:

step(batch_size, ignore_stale_grad=False) method of mxnet.gluon.trainer.Trainer instance
    Makes one step of parameter update. Should be called after
    `autograd.backward()` and outside of `record()` scope.
    
    For normal parameter updates, `step()` should be used, which internally calls
    `allreduce_grads()` and then `update()`. However, if you need to get the reduced
    gradients to perform certain transformation, such as in gradient clipping, then
    you may want to manually call `allreduce_grads()` and `update()` separately.
    
    Parameters
    ----------
    batch_size : int
        Batch size of data processed. Gradient will be normalized by `1/batch_size`.
        Set this to 1 if you normalized loss manually with `loss = mean(loss)`.
    ignore_stale_grad : bool, optional, default=False
        If true, ignores Parameters with stale gradient (gradient that has not
        been updated by `backward` after last 

&**
